# **Интерпретация моделей**

## **Подготовка для работы в Google Colab или Kaggle**

#### Код для подключения Google Drive в Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#### Код для получения пути к файлам в Kaggle

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

#### Код для установки библиотек

In [ ]:
%pip install numpy==1.26.4 pandas==2.1.4 scikit-learn==1.7.0 matplotlib==3.8.0 catboost==1.2.8 eli5==0.16.0 shap==0.48.0

## **Важная информация**

**Для правильного воспроизведения результатов** решения задач:

* Рекомендуется придерживаться имеющего в заданиях кода в исходной последовательности. Для этого при решении задач **восстановите недостающие фрагменты кода, которые отмечены символом** `...` (Ellipsis).

* Если класс, функция или метод предусматривает параметр random_state, всегда указывайте **random_state=RANDOM_STATE**.

* Для всех параметров (кроме random_state) класса, функции или метода **используйте значения по умолчанию, если иное не указано в задании**.

**Если скорость обучения слишком низкая**, рекомендуется следующее:

* В модели или/и GridSearchCV поменяйте значение параметра n_jobs, который отвечает за параллелизм вычислений.

* Воспользуйтесь вычислительными ресурсами Google Colab или Kaggle.

***Использовать GPU не рекомендуется, поскольку результаты обучения некоторых моделей могут отличаться на CPU и GPU.***

После выполнения каждого задания **ответьте на вопросы в тесте.**

**ВНИМАНИЕ:** **После каждого нового запуска ноутбука** перед тем, как приступить к выполнению заданий, проверьте настройку виртуального окружения, выполнив код в ячейке ниже.

In [ ]:
# Код для проверки настройки виртуального окружения

import sys
from importlib.metadata import version

required = {
    'python': '3.11.x',
    'numpy': '1.26.4',
    'pandas': '2.1.4',
    'scikit-learn': '1.7.0',
    'matplotlib': '3.8.0',
    'catboost': '1.2.8',
    'eli5': '0.16.0',
    'shap': '0.48.0'
}

print(f'{"Компонент":<15} | {"Требуется":<12} | {"Установлено":<12} | {"Соответствие"}')
print('-' * 62)

environment_ok = True
for lib, req_ver in required.items():
    try:
        if lib == 'python':
            inst_ver = sys.version.split()[0]
            status = '✓' if sys.version_info.major == 3 and sys.version_info.minor == 11 else f'x (требуется {req_ver})'
        else:
            inst_ver = version(lib)
            if inst_ver == req_ver:
                status = '✓'
            else:
                environment_ok = False
                status = f'x (требуется {req_ver})'
    except:
        environment_ok = False
        inst_ver = '-'
        status = 'x (не установлена)'
    print(f'{lib:<15} | {req_ver:<12} | {inst_ver:<12} | {status:<12}')

print('\nРезультат проверки: ', 
      '✓\nВсе версии соответствуют требованиям' 
      if environment_ok else 
      'x\nВНИМАНИЕ: Версии некоторых компонентов не соответствуют требованиям!\n'
      'Для решения проблемы обратитесь к инструкции по настройке виртуального окружения')

## **Импорт библиотек и вспомогательные функции**

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.inspection import PartialDependenceDisplay
from sklearn.compose import ColumnTransformer, make_column_selector

from catboost import CatBoostClassifier

import eli5
from eli5.sklearn import PermutationImportance

import shap
from shap import TreeExplainer

In [ ]:
RANDOM_STATE = 42

## **Практическая часть**

### **Permutation Importance**

Permutation importance (вычисление важности с помощью перестановки) — это метод интерпретации моделей машинного обучения, используемый для количественной оценки вклада каждого признака в прогноз модели. Метод оценивает, насколько сильно ухудшается метрика качества модели после случайного перемешивания значений одного из признаков.

Если признак  действительно важен для модели, то его случайное перемешивание разрушает взаимосвязь между признаком и целевой переменной, что приводит к заметному снижению метрики (например, R² или MSE). Если признак неважен, то перемешивание не влияет или почти не влияет на качество модели.
                            
**Алгоритм вычисления:**

1. Модель $m$ обучается на обучающей выборке.

2. Вычисляется исходная метрика $s$ модели $m$ на валидационной выборке $D$.

3. Для каждого признака $j$:

    1. Для каждого повторения $k$ ($k=1,...,K$):

        1. Случайным образом перетасовывается столбец $j$ набора данных $D$ для создания нового набора данных $D_{k,j}$.

        2. Вычисляется метрика $s_{k,j}$ модели $m$ по данным $D_{k,j}$.

    2. Вычисляется важность $i_{j}$ для признака $j$ по формуле:
    
    $$i_{j}=s-\frac{1}{K}{\sum_{k=1}^{K}{s_{k,j}}}$$
    
    Чем больше $i_{j}$, тем сильнее вклад признака в качество модели.

**Применение:**

* Признаки с $i_{j}$, близкой к нулю, можно удалить без значимой потери качества модели.

* Значение $i_{j}$ позволяет ранжировать признаки по влиянию на модель.

Подробнее можно изучить по **ссылкам:**

* [Вычисление важности признаков с помощью перестановки | scikit-learn.ru](https://scikit-learn.ru/stable/modules/permutation_importance.html)

* [Permutation Feature Importance |christophm.github.io](https://christophm.github.io/interpretable-ml-book/feature-importance.html)

### **Датасет *Red Wine Quality***

**Для решения задания 1 рассмотрим датасет [Red Wine Quality](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009).**

Датасет содержит информацию о физико-химических свойствах красных вин португальского сорта "Vinho Verde". Он включает в себя результаты лабораторных анализов и оценку качества экспертами.

Целевая переменная — признак quality (качество вина), представляющий собой оценку вина экспертами по шкале от 0 до 10.

Датасет содержит только числовые признаки:

* Кислотность: fixed acidity, volatile acidity, citric acid.

* Содержание сахара и хлоридов: residual sugar, chlorides.

* Уровень диоксида серы: free sulfur dioxide, total sulfur dioxide.

* Физические свойства: density, pH.

* Крепость: alcohol.

*Целевая переменная quality принимает целые значения от 0 до 10, в рамках решения задания предлагается использовать набор данных для решения задачи регрессии (предполагая непрерывность целевой переменной).*

### ***Задание 1***

Выполните предобработку датасета (см. код).

Обучите две модели:

* `sk_reg_wine` — линейная регрессия (sklearn LinearRegression).

* `rf_wine` — случайный лес (RandomForestRegressor) с n_estimators=100.

Оцените важность признаков с точки зрения:

* Абсолютных значений ('по модулю') коэффициентов регрессии `sk_reg_wine`.

* Оценок относительного вклада признаков случайного леса `rf_wine` ([RandomForestRegressor.feature_importances_](https://scikit-learn.org/stable/auto_examples/ensemble/plot_forest_importances.html#feature-importance-based-on-mean-decrease-in-impurity)).

* Значений permutation importance для `sk_reg_wine` **на валидационной выборке**.

* Значений permutation importance для `rf_wine` **на валидационной выборке**.

**ВНИМАНИЕ:** В рамках данного задания признаки масштабируются с помощью `wine_scaler` (StandardScaler). Для интерпретации коэффициентов линейной регрессии и важности признаков **осуществлять обратное преобразование не нужно**.

In [ ]:
# Считайте набор данных

df_wine = pd.read_csv('wine.csv')
df_wine

In [ ]:
# Воспользуемся методом info для определения типов признаков

df_wine.info()

In [ ]:
# Воспользуемся методом nunique для определения числа уникальных значений в каждом из признаков

df_wine.nunique()

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_wine, y_wine = ...

In [ ]:
# Разделите датасет на обучающую (60%) и валидационную (40%) выборки
# Не забудьте зафиксировать RANDOM_STATE

X_wine_train, X_wine_val, y_wine_train, y_wine_val = ...

In [ ]:
# Масштабируйте все признаки
#   train -> fit_transform
#   test -> transform

wine_scaler = StandardScaler().set_output(transform='pandas')

X_wine_train = ...
X_wine_val = ...

In [ ]:
# Обучите sk_reg_wine (sklearn LinearRegression) на обучающей выборке

sk_reg_wine = ...

In [ ]:
# Оцените важность признаков sk_reg_wine с помощью permutation importance на валидационной выборке
# Не забудьте зафиксировать RANDOM_STATE

n_iter = 5

perm_sk_reg_wine = PermutationImportance(
    estimator=...,
    n_iter=...,
    random_state=...
)
...
eli5.show_weights(perm_sk_reg_wine, feature_names=...)

In [ ]:
# Обучите rf_wine (RandomForestRegressor) на обучающей выборке с n_estimators=100
# Не забудьте зафиксировать RANDOM_STATE

rf_wine = ...

In [ ]:
# Оцените важность признаков rf_wine с помощью permutation importance на валидационной выборке
# Не забудьте зафиксировать RANDOM_STATE

n_iter = 5

perm_rf_wine = ...

eli5.show_weights(...)

In [ ]:
# Создайте DataFrame с оценками важности признаков с точки зрения:
#   1. Абсолютных значений ('по модулю') коэффициентов регрессии sk_reg_wine
#   2. Оценок относительного вклада признаков случайного леса rf_wine (RandomForestRegressor.feature_importances_)
#   3. Значений permutation importance для sk_reg_wine на валидационной выборке
#   4. Значений permutation importance для rf_wine на валидационной выборке

wine_feat_importance = pd.DataFrame({
    'sk_reg_wine coef': ...,
    'rf_wine importance': ...,
    'sk_reg_wine permutation importance': ...,
    'rf_wine permutation importance': ...
},
index = list(X_wine_val.columns))
wine_feat_importance = wine_feat_importance.sort_values(by='sk_reg_wine coef')
wine_feat_importance

In [ ]:
# Визуализируйте оценки важности признаков

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0, 0].barh(wine_feat_importance.index, wine_feat_importance['sk_reg_wine coef'], color='skyblue')
axes[0, 0].set_title('sk_reg_wine coef')

axes[0, 1].barh(wine_feat_importance.index, wine_feat_importance['rf_wine importance'], color='lightgreen')
axes[0, 1].set_title('rf_wine importance')

axes[1, 0].barh(wine_feat_importance.index, wine_feat_importance['sk_reg_wine permutation importance'], color='salmon')
axes[1, 0].set_title('sk_reg_wine permutation importance')

axes[1, 1].barh(wine_feat_importance.index, wine_feat_importance['rf_wine permutation importance'], color='gold')
axes[1, 1].set_title('rf_wine permutation importance')

plt.tight_layout()
plt.show()

### **Partial Dependence Plot**

Partial dependence plot (PDP, график частичной зависимости) — это метод интерпретации моделей машинного обучения, позволяющий визуализировать среднее влияние признаков на предсказания модели, усредняя влияние остальных признаков. PDP показывает, как в среднем изменяется прогноз модели при варьировании значений выбранного признака при неизменных значениях остальных признаков.

**Алгоритм вычисления:**

Пусть необходимо оценить PD для признака $S$.

1. Модель $m$ обучается на обучающей выборке.
    
2. Для каждого уникального значения $v$ (или для равномерно распределенной сетки) признака $S$:

    1. Все значения признака $S$ в валидационном датасете $D$ заменяются на $v$ для всех наблюдений (остальные признаки остаются неизменными) для создания нового набора данных $D_v$.

    2. Вычисляется : $\widehat{y}_i$.

    3. Вычисляется $PD(v)$:
    
    $$PD(v)=\frac{1}{N}\sum_{i=1}^{N}{\widehat{y}_i}$$
      
    где $\widehat{y}_i$ — прогноз модели $m$ на $D_v$ для наблюдения $i$.

    Полученное значение $PD(v)$ — partial dependence для значения $v$ признака $S$.

**Применение:**

* Визуализация характера влияния признака на прогноз модели:

    * Если кривая PDP почти горизонтальна, признак слабо влияет на предсказание.

    * Если PDP монотонно возрастает (или убывает), увеличение признака связано с ростом (или снижением) прогноза.
    
    * Если PDP нелинейный, то наблюдается сложная зависимость между признаком и целевой переменной.

* PDP используется для сравнения того, как разные модели интерпретируют влияние одного и того же признака.

Подробнее можно изучить по **ссылкам:**

* [Графики частичной зависимости (Partial dependence plots - PDP) | scikit-learn.ru](https://scikit-learn.ru/stable/modules/partial_dependence.html#partial-dependence-plots-pdp)

* [Partial Dependence Plot (PDP) | christophm.github.io](https://christophm.github.io/interpretable-ml-book/pdp.html)

### **Датасет *Hotel Reservations Dataset***

**Для решения заданий 2 — 4 рассмотрим датасет [Hotel Reservations Dataset](https://www.kaggle.com/datasets/ahsan81/hotel-reservations-classification-dataset).**

**ВНИМАНИЕ:** При решении заданий **используйте файл hotels.csv** из приложения к ноутбуку, поскольку исходный датасет был изменен авторами курса.

Датасет содержит записи бронирования комнат в двух португальских отелях розничными и корпоративными клиентами, включая детали бронирования и статус бронирования.

Целевая переменная — booking_status (статус бронирования):

* Canceled — бронь была отменена клиентами до заселения.

* Not_Canceled — бронь не была отменена, клиенты заселились в отель.

Датасет содержит признаки:

* Количество гостей: no_of_adults, no_of_children.

* Время проживания: no_of_nights.

* Количество суток между бронированием и заселением: lead_time.

* Средняя стоимость аренды номера (евро): avg_price_per_room.

* Тип питания: type_of_meal_plan.

* Тип номера: room_type_reserved.

* Сегмент рынка: market_segment_type.

* Требуется ли парковка: required_car_parking_space.

* Количество специальных услуг: no_of_special_requests.

* Дата заселения: arrival_year, arrival_month, arrival_date.

* История клиента: repeated_guest, no_of_previous_cancellations, no_of_previous_bookings_not_canceled.

### ***Задание 2***

Выполните предобработку датасета (см. код).

Обучите две модели, предварительно подобрав оптимальные гиперпараметры обучения для каждой их них с помощью GridSearchCV:

* `lr_hotels` — логистическая регрессия (LogisticRegression). Для обучения и валидации используйте `X_hotels_train_scaled` и `X_hotels_val_scaled`.

* `rf_hotels` —  случайный лес (RandomForestClassifier) с n_estimators=100. Для обучения и валидации используйте `X_hotels_train_scaled` и `X_hotels_val_scaled`.

Для моделей `lr_hotels` и `rf_hotels` постройте отчёты по метрикам классификации и рассчитайте AUC **на валидационной выборке**.

Оцените важность признаков для моделей `lr_hotels` и `rf_hotels` с точки зрения permutation importance **на валидационной выборке (метрика — 'roc_auc')** и выделите **два наиболее важных признака** для `rf_hotels` — `hotels_important_features`.

Постройте partial dependence plot (PDP) признаков `hotels_important_features` для моделей `lr_hotels` и `rf_hotels` **на валидационной выборке**.

In [ ]:
# Считайте набор данных

df_hotels = pd.read_csv('hotels.csv')
df_hotels

In [ ]:
# Воспользуемся методом info для определения типов признаков

df_hotels.info()

In [ ]:
# Воспользуемся методом nunique для подсчета количества уникальных значений переменных в датасете

df_hotels.nunique()

In [ ]:
# Рассмотрим соотношение долей классов в датасете

df_hotels['booking_status'].value_counts(normalize=True)

In [ ]:
# Выделите объясняемый фактор в отдельную переменную

X_hotels, y_hotels = ...

In [ ]:
# Закодируйте объясняемый фактор как бинарную переменную:
#   Not_Canceled — 0
#   Canceled — 1

y_hotels = ...

In [ ]:
# Создайте списки количественных и категориальных переменных (не включая целевую переменную)

hotels_num_feat = ['no_of_nights', 'no_of_adults', 'no_of_children', 'lead_time', 'no_of_previous_cancellations', 'no_of_previous_bookings_not_canceled', 'avg_price_per_room', 'no_of_special_requests']
hotels_cat_feat = ['type_of_meal_plan', 'required_car_parking_space', 'room_type_reserved', 'market_segment_type', 'repeated_guest', 'arrival_year', 'arrival_month', 'arrival_date']

In [ ]:
# Разделите датасет на обучающую (60%) и валидационную (40%) выборки со стратификацией по целевой переменной
# Выполните сброс индексов в полученных датасетах
# Не забудьте зафиксировать RANDOM_STATE

X_hotels_train, X_hotels_val, y_hotels_train, y_hotels_val = ...

X_hotels_train = X_hotels_train.reset_index(drop=True)
y_hotels_train.index = X_hotels_train.index
X_hotels_val = X_hotels_val.reset_index(drop=True)
y_hotels_val.index = X_hotels_val.index

In [ ]:
# Закодируйте категориальные признаки числами 0 и 1 с помощью OneHotEncoder
# Выделите отдельные датасеты с закодированными признаками
#   train -> fit_transform
#   val -> transform

hotels_encoder = OneHotEncoder(sparse_output=False, drop='first').set_output(transform='pandas')

X_hotels_train_onehot = ...
X_hotels_val_onehot = ...

In [ ]:
# Масштабируйте количественные признаки
#   train -> fit_transform
#   val -> transform

hotels_scaler = StandardScaler().set_output(transform='pandas')

X_hotels_train_onehot_scaled = ...
X_hotels_val_onehot_scaled = ...

hotels_onehot_scaled_cols = list(X_hotels_train_onehot_scaled.columns)

In [ ]:
# Подберите оптимальные гиперпараметры обучения lr_hotels с помощью GridSearchCV
# Обучите lr_hotels с оптимальными параметрами
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'C': [0.001, 0.01, 0.05, 0.1, 0.5, 1.0, 5.0]
}
scoring = 'roc_auc'
cv = 5

cv_lr_hotels = ...

lr_hotels = ...

In [ ]:
# Постройте отчёт по метрикам классификации для lr_hotels на валидационной выборке

...

In [ ]:
# Рассчитайте AUC для lr_hotels на валидационной выборке

...

In [ ]:
# Оцените важность признаков для lr_hotels с точки зрения permutation importance на валидационной выборке (метрика — 'roc_auc')

n_iter = 5
scoring = 'roc_auc'

perm_lr_hotels = ...
eli5.show_weights(..., top=len(hotels_onehot_scaled_cols))

In [ ]:
# Подберите оптимальные гиперпараметры обучения rf_hotels (RandomForestClassifier) с помощью GridSearchCV
# Обучите rf_hotels с оптимальными параметрами
# Не забудьте зафиксировать RANDOM_STATE

params = {
    'max_depth': [None, 6, 7, 8, 9],
    'n_estimators': [50, 100, 150]
}
scoring = 'roc_auc'
cv = 5

cv_rf_hotels = ...

rf_hotels = ...

In [ ]:
# Постройте отчёт по метрикам классификации для rf_hotels на валидационной выборке

...

In [ ]:
# Рассчитайте AUC для rf_hotels на валидационной выборке

...

In [ ]:
# Оцените важность признаков для rf_hotels с точки зрения permutation importance на валидационной выборке (метрика — 'roc_auc')

n_iter = 5
scoring = 'roc_auc'

perm_rf_hotels = ...

eli5.show_weights(...)

In [ ]:
# Выделите 2 наиболее важных признака для rf_hotels с точки зрения permutation importance в список

hotels_important_features = ...

In [ ]:
# Постройте PDP признаков hotels_important_features для lr_hotels на валидационной выборке

fig, ax = plt.subplots(figsize=(20, 6))
PartialDependenceDisplay.from_estimator(
    estimator=..., 
    X=..., 
    features=hotels_important_features,
    centered=True,
    ax=ax,
    kind='both',
    pd_line_kw={'color': 'red'}
)
ax.plot()

In [ ]:
# Постройте PDP признаков hotels_important_features для rf_hotels на валидационной выборке

fig, ax = plt.subplots(figsize=(20, 6))
PartialDependenceDisplay.from_estimator(
    estimator=..., 
    X=..., 
    features=hotels_important_features,
    centered=True,
    ax=ax,
    kind='both',
    pd_line_kw={'color': 'red'}
)
ax.plot()

### **SHAP**

SHAP (SHapley Additive exPlanations) — это метод интерпретации моделей машинного обучения, объясняющий предсказание для каждого отдельного объекта как сумму аддитивных вкладов его признаков. 

Метод основан на значениях Шепли — концепции из кооперативной теории игр, которая позволяет справедливо распределить "выигрыш" (в данном случае, значение прогноза модели) между "игроками" (признаками). 

Идея метода состоит в оценке вклада каждого признака в отклонение конкретного предсказания от среднего предсказания модели по всей выборке. 

SHAP позволяет понять вклад каждого признака в конкретный прогноз.  Усреднение SHAP-вкладов по всем объектам даёт глобальную характеристику важности признаков.

**Применение:**

* Force Plot — визуализация важности признаков для отдельного предсказания, визуально показывающее вклад каждого признака в итоговое значение.

* Summary Plot — сводная визуализация вклада признаков по всему датасету, показывающая распределение и важность признаков.

* Dependence Plot — график зависимости вклада признака от его значения, позволяющий выявить нелинейные эффекты и взаимодействия.

Подробнее можно изучить по **ссылкам:**

* [An introduction to explainable AI with Shapley values | shap.readthedocs.io](https://shap.readthedocs.io/en/latest/example_notebooks/overviews/An%20introduction%20to%20explainable%20AI%20with%20Shapley%20values.html)

* [SHAP | christophm.github.io](https://christophm.github.io/interpretable-ml-book/shap.html)

### ***Задание 3***

**ВНИМАНИЕ:** Для решения этого задания используйте:

* Обучающую и валидационную выборки из задания 2 **до One-Hot кодирования и масштабирования**: `X_hotels_train`, `y_hotels_train`, `X_hotels_val`, `y_hotels_val`.

Обучите модель `catb_hotels` (CatBoostClassifier) c параметрами по умолчанию.

Постройте отчёт по метрикам классификации для модели и рассчитайте AUC **на валидационной выборке** для модели `catb_hotels`.

**Используя валидационную выборку**, вычислите SHAP-значения с помощью [TreeExplainer](https://shap.readthedocs.io/en/latest/generated/shap.TreeExplainer.html): `hotels_shap_values`.

Визуализуйте и интерпретируйте глобальную важность признаков, построив для `hotels_shap_values` графики [barplot](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/bar.html#Global-bar-plot) и [beeswarm](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/beeswarm.html#beeswarm-plot).

In [ ]:
# Обучите модель catb_hotels (CatBoostClassifier) c параметрами по умолчанию (и params)
# Для обучения и валидации используйте X_hotels_train и X_hotels_val

params = {
    'cat_features': hotels_cat_feat,
    'random_state': RANDOM_STATE,
    'verbose': False
}

catb_hotels = ...

In [ ]:
# Общее количество признаков (для визуализации shap)

hotels_feat_count = len(X_hotels_train.columns)

In [ ]:
# Постройте отчёт по метрикам классификации для catb_hotels на валидационной выборке

...

In [ ]:
# Рассчитайте AUC для catb_hotels на валидационной выборке

...

In [ ]:
# Вычислите SHAP-значения для валидационной выборки, используя TreeExplainer

hotels_shap_explainer = TreeExplainer(...)
hotels_shap_values = hotels_shap_explainer(...)

In [ ]:
# Постройте barplot SHAP-значений

shap.plots.bar(hotels_shap_values, max_display=hotels_feat_count)

In [ ]:
# Постройте beeswarm SHAP-значений

shap.plots.beeswarm(...)

### ***Задание 4***

**ВНИМАНИЕ:** Для решения этого задания используйте:

* Валидационную выборка из задания 2 **до One-Hot кодирования и масштабирования**: `X_hotels_val`, `y_hotels_val`.

* SHAP-значения из задания 3: `hotels_shap_values`.

В рамках валидационной выборки `X_hotels_val` выделите три группы клиентов отелей по признакам (используйте метод [query](https://pandas.pydata.org/docs/user_guide/indexing.html#the-query-method)):

* **Группа 1.** Брони семей с детьми (количество взрослых — больше одного, количество детей — как минимум один) в недорогих номерах (средняя стоимость аренды номера строго меньше 90 евро).

* **Группа 2.** Брони новых (repeated_guest — 0) корпоративных клиентов (сегмент рынка — "Corporate") на одну ночь (время пребывания — одна ночь).

* **Группа 3.** Брони клиентов, которые забронировали номер через оффлайн-канал (сегмент рынка — "Offline") на срок более пяти ночей (время пребывания — строго больше 5 ночей).

Визуализуйте и интерпретируйте важность признаков для каждой из выделенных групп, построив для SHAP-значений [beeswarm](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/beeswarm.html#beeswarm-plot) график.

Используя метод [sample](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html), для каждой из выделенных групп выберите по одной случайной записи (**не забудьте зафиксировать RANDOM_STATE**):

* Случайная бронь из группы 1. 

* Случайная бронь из группы 2. 

* Случайная бронь из группы 3. 

Визуализуйте и интерпретируйте важность признаков для каждой из трех случайных записей, построив для SHAP-значений графики: [waterfall](https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/waterfall.html#waterfall-plot) и [force plot](https://shap.readthedocs.io/en/latest/generated/shap.plots.force.html).

##### Группа 1

In [ ]:
# Из валидационной выборки выделите группу 1
# Подсказка: используйте метод query

X_hotels_group_1 = X_hotels_val.query(...)

In [ ]:
# Постройте beeswarm SHAP-значений для группы 1

shap.plots.beeswarm(hotels_shap_values[...], max_display=hotels_feat_count)

In [ ]:
# Выберите случайную запись из группы 1 
# Не забудьте зафиксировать RANDOM_STATE

X_hotels_group_1_sample = X_hotels_group_1.sample(...)

In [ ]:
# Постройте waterfall SHAP-значений для записи из группы 1

shap.plots.waterfall(hotels_shap_values[...], max_display=hotels_feat_count)

In [ ]:
# Постройте force plot SHAP-значений для записи из группы 1

shap.plots.force(hotels_shap_values[...], matplotlib=True)

##### Группа 2

In [ ]:
# Из валидационной выборки выделите группу 2
# Подсказка: используйте метод query

X_hotels_group_2 = ...

In [ ]:
# Постройте beeswarm SHAP-значений для группы 2

shap.plots.beeswarm(...)

In [ ]:
# Выберите случайную запись из группы 2
# Не забудьте зафиксировать RANDOM_STATE

X_hotels_group_2_sample = ...

In [ ]:
# Постройте waterfall SHAP-значений для записи из группы 2

shap.plots.waterfall(...)

In [ ]:
# Постройте force plot SHAP-значений для записи из группы 2

shap.plots.force(..., matplotlib=True)

##### Группа 3

In [ ]:
# Из валидационной выборки выделите группу 3
# Подсказка: используйте метод query

X_hotels_group_3 = ...

In [ ]:
# Постройте beeswarm SHAP-значений для группы 3

shap.plots.beeswarm(...)

In [ ]:
# Выберите случайную запись из группы 3
# Не забудьте зафиксировать RANDOM_STATE

X_hotels_group_3_sample = ...

In [ ]:
# Постройте waterfall SHAP-значений для записи из группы 3

shap.plots.waterfall(...)

In [ ]:
# Постройте force plot SHAP-значений для записи из группы 3

shap.plots.force(..., matplotlib=True)